In [93]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, KBinsDiscretizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

from feature_engine import outlier_removers 
from feature_engine.categorical_encoders import OneHotCategoricalEncoder, RareLabelCategoricalEncoder

# Display options

pd.options.mode.chained_assignment = None #set it to None to remove SettingWithCopyWarning
pd.options.display.float_format = '{:.4f}'.format #set it to convert scientific noations such as 4.225108e+11 to 422510842796.00
pd.set_option('display.max_columns', 100) #  display all the columns
pd.set_option('display.max_rows', 100) # display all the rows
np.set_printoptions(suppress=True,formatter={'float_kind':'{:f}'.format})


# Loading Data

In [94]:
df = pd.read_csv('Data/loan.csv',low_memory=False)
print('df is loaded')

# the list contains features that are either proven useless or introduce look-ahead bias into data.
list_to_remove = ['last_pymnt_amnt','total_rec_prncp','total_pymnt',
                  'total_pymnt_inv','total_rec_int','total_rec_late_fee','total_rec_prncp',
                  'issue_d','earliest_cr_line','last_pymnt_d',
                  'last_credit_pull_d','id','member_id','settlement_date',
                  'next_pymnt_d','zip_code',
                  
                 'delinq_2yrs','mths_since_last_delinq', 'pub_rec', 'recoveries', 'collection_recovery_fee',
 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'open_acc_6m', 'open_act_il', 'open_il_12m',
 'open_il_24m','mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
 'all_util','inq_fi', 'total_cu_tl', 'inq_last_12m', 'chargeoff_within_12_mths', 'delinq_amnt',
 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_tl_120dpd_2m', 'num_tl_30dpd', 
                  'num_tl_90g_dpd_24m', 'pub_rec_bankruptcies', 'tax_liens']

df.drop(list_to_remove,axis='columns',inplace=True)

df = df.infer_objects()
# drop any features that have more than 30% of NaN values in them.
df.dropna(axis=1,how='any',thresh=int(0.3*len(df)),inplace=True)
print(df.shape)

df is loaded
(2260668, 61)


# Label Manipulations
* Specific to the dataset

In [118]:
labels_to_drop = ['Current','Late (31-120 days)','Late (16-30 days)','In Grace Period','Default']
df = df[~df.loan_status.isin(labels_to_drop)]

dictionary = {'Does not meet the credit policy. Status:Fully Paid':'Fully Paid',
             'Does not meet the credit policy. Status:Charged Off':'Charged Off'}

df['loan_status'].replace(dictionary,inplace=True)
df['loan_status'].value_counts(normalize=True)

Fully Paid    0.7991
Charged Off   0.2009
Name: loan_status, dtype: float64

In [119]:
# delete all the columns that contain single unique values
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
print(df.shape)        

(1306356, 57)


# Train test split

In [120]:
y = df[['loan_status']].values.ravel()
X = df.drop('loan_status',axis='columns')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                            random_state=42, stratify=None)

# Dividing data into categorical and numerical parts

In [121]:
# dividing training and testing data into categorical and numerical parts
nmrcl_X_train = X_train.select_dtypes(exclude=['object'])
nmrcl_X_test = X_test.select_dtypes(exclude=['object'])

ctgrcl_X_train = X_train.select_dtypes(include=['object'])
ctgrcl_X_test = X_test.select_dtypes(include=['object'])


print('Numerical part:')
print(nmrcl_X_train.shape)
print(nmrcl_X_test.shape)
print('Categorical part:')
print(ctgrcl_X_train.shape)
print(ctgrcl_X_test.shape)

Numerical part:
(979767, 41)
(326589, 41)
Categorical part:
(979767, 15)
(326589, 15)


# Treating numerical data
* starting with pd.fillna

In [122]:
""" 
Training df medians have to be saved as a pd.Series object othervise replace() 
method does not work when replacing NaN in testing df.
"""

training_medians = pd.Series(nmrcl_X_train.median()) # get the training medians 

nmrcl_X_train = nmrcl_X_train.fillna(training_medians) # fillna first
nmrcl_X_test = nmrcl_X_test.fillna(training_medians)

# Pipelining numerical features treatment
* I actually don't know if it's a good idea, but if we pipeline, we decrease the number of points of failure.

In [123]:
# 1.outlier replacement
# 2.discretization
# 3.scaling

capper = outlier_removers.Winsorizer(distribution='skewed', tail='both', fold=1.5)
discretizer = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='kmeans')
scaler = MinMaxScaler()

numerical_pipeline = Pipeline([('capper',capper),
                    ('discretizer',discretizer),
                    ('scaler',scaler)])

nmrcl_X_train = numerical_pipeline.fit_transform(nmrcl_X_train)
nmrcl_X_test = numerical_pipeline.transform(nmrcl_X_test)

# Cast produced np.arrays back to pd.DataFrame

In [124]:
# this is the way to access names of the columns, it's needed to convert 
# pipeline-produced np.array back to pd.DataFrame.
nmrc_feature_cols = numerical_pipeline.named_steps['capper'].variables

nmrcl_X_train = pd.DataFrame(nmrcl_X_train, columns=nmrc_feature_cols)
nmrcl_X_test = pd.DataFrame(nmrcl_X_test, columns=nmrc_feature_cols)

# Pipelining categorical features treatment

In [125]:
ctgrcl_X_train.fillna('other',inplace=True)
ctgrcl_X_test.fillna('other',inplace=True)

# two step pipeline:
# 1. rare labels (frequency below 1% are changed to 'rare')
# 2. n-1 OneHot encoding

encoder = RareLabelCategoricalEncoder(tol=0.01)
ohe_enc = OneHotCategoricalEncoder(top_categories=None,drop_last=True)

categorical_pipeline = Pipeline([('rare_label',encoder),('onehot',ohe_enc)])

ctgrcl_X_train = categorical_pipeline.fit_transform(ctgrcl_X_train)
ctgrcl_X_test = categorical_pipeline.transform(ctgrcl_X_test)

# reseting the index so all the dfs are alinable
ctgrcl_X_train.reset_index(drop=True,inplace=True)
ctgrcl_X_test.reset_index(drop=True,inplace=True)

# Label Encoding 

In [126]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

# Cast encoded labels back to a dataframe object

In [127]:
"""
LabelEncoder() output is a numpy array, it's missing the index which is later used for
concatanation of categorical, numercial and label data together. The following is a 
primitive solution but it works and there is no missalignment in the final df.

"""
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

y_train.columns = ['training labels']
y_test.columns = ['testing labels']

# Stacking all the dataframes together

In [128]:
final_train = pd.concat([nmrcl_X_train,ctgrcl_X_train,y_train],axis=1)
final_test = pd.concat([nmrcl_X_test,ctgrcl_X_test,y_test],axis=1)

print(final_train.shape)
print(final_test.shape)

(979767, 143)
(326589, 143)


In [129]:
final_train.to_csv('Data/2_bins_train.csv')
final_test.to_csv('Data/2_bins_test.csv')